In [22]:
import re

In [37]:
with open('verdict.txt',"r",encoding="utf-8") as f:
    txt=f.read()

In [38]:
#lets tokenize 
tokenized=re.split(r'([,.?_!"()\']|--|\s)',txt)

# print(len(tokenized))
tokenized=[item for item in tokenized if item.split()]
print(len(tokenized))


4649


# lets make vocublary

In [41]:
clean_token=sorted(list(set(tokenized)))

In [46]:
print(len(clean_token))

1159


In [53]:
#lets make the vocab
text_index={text:index for index,text in enumerate(clean_token)}
index_text={index:text for index,text in enumerate(clean_token)}

In [52]:
print(text_index['that'])

1012


In [58]:
#this is the encoder
def encoder(text):
    #lets tokenize it
    tokenizer=re.split(r'([,.?_!"()\']|--|\s)',text)  
    tokenizer=[item for item in tokenizer if item.strip()]
    
    #lets make token id
    token_id=[]
    for i in tokenizer:
        token_id.append(text_index[i])
    return token_id

#this is the decoder
def decoder(encoded):
    decoded_id=[]
    for i in encoded:
        decoded_id.append(index_text[i])
    return decoded_id

In [60]:
print(encoder('this is not good'))
print(decoder([1024, 595, 727, 508]))

[1024, 595, 727, 508]
['this', 'is', 'not', 'good']


# lets make ita a  class

In [90]:
class TokenizerVersion1():
    def __init__(self,vocab) -> None:
        self.vocab=vocab
    
    def encoder(self,text):
        #lets tokenize it
        tokenizer=re.split(r'([,.?_!"()\']|--|\s)',text)  
        tokenizer=[item for item in tokenizer if item.strip()]
        
        #lets make token id
        token_id=[]
        for i in tokenizer:
            token_id.append(self.vocab[i])
        return token_id
    
    def decoder(self,encoded):
         self.decoded_id=[]
         for i in encoded:
            self.decoded_id.append(index_text[i])
        
         text=" ".join(i for i in self.decoded_id)
         text= re.sub(r'\s+([,.?_!"()\'"])', r'\1', text)
         return text

In [91]:
#lets check out the class
text="look where are they and ?"
toknizer=TokenizerVersion1(vocab=text_index)

encode=toknizer.encoder(text=text)
print(encode)

decoder=toknizer.decoder(encoded=encode)
print(decoder)

[657, 1119, 174, 1019, 162, 10]
look where are they and?
